In [9]:
import numpy as np
import tensorflow.keras
import torch
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Conv1D, AveragePooling1D, Conv2D, MaxPooling2D,ReLU
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model #save and load models
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, CSVLogger, ModelCheckpoint
import IPython.display as ipd
from kymatio import Scattering1D
import hitdifferentparts
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import pescador
import random
import os
import librosa

In [10]:
tf.config.experimental.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [11]:
# scattering order one
def getsc_new(y,J,Q_num,order):
    """
    this function outputs scattering transform of a time-domain signal.
    """
    N = len(y)
    scattering = Scattering1D(J = J,shape=(N,), Q = Q_num, max_order=order)
    Sy = scattering(torch.Tensor(y))
    return Sy

In [12]:
def feature_sampler(df,params_normalized,idx,path_to_folder,J,Q,order):
    """
    output a {input, ground truth} pair for the designated audio sample
    """
    i=idx
    y=np.array(params_normalized[i,:]).reshape((5,)) #df.values[i,1:-1]
    path_to_audio = os.path.join(path_to_folder,str(df.values[i,0])+"_sound.wav") 
    x,fs=librosa.load(path_to_audio)
    Sy = getsc_new(x,J,Q,order)
    m,n = Sy.shape
    Sy2 = np.array(Sy).reshape((n,m))
    
    while True:
        yield {'input': Sy2,'y': y}

        
def data_generator(df, params_normalized, path_to_folder, J, Q, order, batch_size, idx, active_streamers,
                        rate, random_state=12345678):
    """
    use streamers to output a batch of {input groundtruth} pairs. 
    """
    seeds = []
    for i in idx:
        streamer = pescador.Streamer(feature_sampler, df, params_normalized, i,path_to_folder,J,Q,order)
        seeds.append(streamer)

    # Randomly shuffle the seeds
    random.shuffle(seeds)

    mux = pescador.StochasticMux(seeds, active_streamers, rate=rate, random_state=random_state)
   
    if batch_size == 1:
        return mux
    else:
        return pescador.maps.buffer_stream(mux, batch_size)



In [13]:
def create_model(J,Q,order,path_to_train):
    fname = random.choice(os.listdir(path_to_train))
    rand_audio = os.path.join(path_to_train,fname)
    y,sr = librosa.load(rand_audio)
    Sy = getsc_new(torch.Tensor(y),J,Q,order).T
    nrow, ncol = Sy.shape 
    naudio = batch_size         # number of images in batch
    nchan_in = 1       # number of input channels.  1 since it is BW
    #input_shape = (nrow,ncol,nchan_in)  # input shape of 1 image
    input_shape = Sy.shape
    batch_shape = (naudio,nrow,ncol,nchan_in)  # shape of image batch
    #x = Sy.reshape(batch_shape)
    kernel_size = (8,)
    nchan_out = 16

    K.clear_session()
    model=Sequential()
    model.add(Conv1D(input_shape=input_shape, filters=nchan_out,
                     kernel_size=kernel_size,activation= "relu", padding="same",name='conv1'))
    model.add(AveragePooling1D(pool_size=(4,)))
    model.add(Conv1D(filters=16,
                     kernel_size=kernel_size,activation= "relu", padding="same",name='conv2' ))
    model.add(AveragePooling1D(pool_size=(4,)))
    model.add(Conv1D(filters=16,
                     kernel_size=kernel_size,activation= "relu", padding="same",name='conv3' ))
    model.add(AveragePooling1D(pool_size=(4,)))
    model.add(Conv1D(filters=16,
                     kernel_size=kernel_size,activation= "relu", padding="same",name='conv4' ))
    model.add(AveragePooling1D(pool_size=(2,)))

    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    #what activation should be chosen for last layer, for regression problem? should be a linear function
    model.add(Dense(5, activation='linear')) #output layer that corresponds to the 5 physical parameters.


    # Compile the model
    model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
    
    print(model.summary())
    return model

In [29]:
def make_generator():
    df_train = pd.read_csv("./train_param.csv")
    df_test = pd.read_csv("./test_param.csv")
    df_val = pd.read_csv("./val_param.csv")
    df_full = pd.read_csv("./diffshapes_param.csv")
    
    # normalization of the physical parameters
    params = df_train.values[:,1:-1]
    scaler = MinMaxScaler()
    scaler.fit(params)
    train_params_normalized = scaler.transform(params)
    
    train_idx = np.arange(0,params.shape[0],1)#np.arange(0,1000,1) #df_train.values[:1000,0]
    test_idx = np.arange(0,df_test.values.shape[0],1) #df_test.values[:300,0]
    val_idx = np.arange(0,df_val.values.shape[0],1)

   
    test_params_normalized = scaler.transform(df_test.values[:,1:-1])
    val_params_normalized = scaler.transform(df_val.values[:,1:-1])
    train_batches=data_generator(df_train,train_params_normalized, path_to_train,J, Q, order, batch_size, train_idx,active_streamers,rate=64,random_state=random_state)
    test_batches=data_generator(df_test,test_params_normalized, path_to_test,J, Q, order, batch_size, test_idx,active_streamers,rate=64,random_state=random_state)
    val_batches = data_generator(df_val,val_params_normalized, path_to_val,J, Q, order, batch_size, val_idx,active_streamers,rate=64,random_state=random_state)
    steps_per_epoch = len(train_idx) // batch_size
    train_gen = pescador.maps.keras_tuples(train_batches, 'input', 'y')
    test_gen = pescador.maps.keras_tuples(test_batches, 'input', 'y')
    val_gen = pescador.maps.keras_tuples(val_batches, 'input', 'y')
    return train_gen,test_gen,val_gen,scaler,steps_per_epoch


In [22]:
import pickle
def get_val_from_pickle(scaler):
    pkl_path = '/scratch/hh2263/drum_data/val/J_8_Q_1_order_2.pkl'
    pkl_file = open(pkl_path, 'rb')
    Sy_val,y_val = pickle.load(pkl_file) 
    Sy_val = Sy_val.reshape((Sy_val.shape[2],Sy_val.shape[0],Sy_val.shape[1]))
    y_val = y_val.astype('float32')
    y_val_normalized = scaler.transform(y_val)
    return Sy_val,y_val_normalized

In [26]:
def fit_model(epochs,train_gen,Sy_val,y_val_normalized,steps_per_epoch):
    output_dir = "../output/"

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    model_filepath = os.path.join(output_dir, "training_1/cp.ckpt")
    log_filepath = os.path.join(output_dir, 'train_log.csv')

    callbacks = []
    callbacks.append(EarlyStopping(patience=10))
    callbacks.append(ModelCheckpoint(model_filepath, save_best_only=True))
    callbacks.append(CSVLogger(log_filepath))
    
    for epoch in range(epochs):
        model.fit(train_gen,
                  steps_per_epoch=steps_per_epoch/5,
                  epochs=1,
                  use_multiprocessing=True,
                  callbacks = callbacks,
                  verbose=1,

                 )
        print('done fitting')
        loss,accuracy = model.evaluate(Sy_val,y_val_normalized)
        #print(loss,accuracy)


In [18]:
epochs=12
batch_size=32
random_state=12345678
active_streamers=64
J = 8
Q = 1
order = 2 # remember to go to order 2 eventually
path_to_train = "/scratch/hh2263/drum_data/train/"
path_to_test = "/scratch/hh2263/drum_data/test/"
path_to_val = "/scratch/hh2263/drum_data/val"

model = create_model(J,Q,order,path_to_train)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv1D)               (None, 128, 16)           5520      
_________________________________________________________________
average_pooling1d (AveragePo (None, 32, 16)            0         
_________________________________________________________________
conv2 (Conv1D)               (None, 32, 16)            2064      
_________________________________________________________________
average_pooling1d_1 (Average (None, 8, 16)             0         
_________________________________________________________________
conv3 (Conv1D)               (None, 8, 16)             2064      
_________________________________________________________________
average_pooling1d_2 (Average (None, 2, 16)             0         
_________________________________________________________________
conv4 (Conv1D)               (None, 2, 16)             2

In [30]:
train_gen,test_gen,val_gen,scaler,steps_per_epoch =make_generator()
Sy_val,y_val_normalized = get_val_from_pickle(scaler)

In [33]:
output_dir = "../output/"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)
model_filepath = os.path.join(output_dir, "training_1/cp.ckpt")
log_filepath = os.path.join(output_dir, 'train_log.csv')

callbacks = []
callbacks.append(EarlyStopping(patience=10))
callbacks.append(ModelCheckpoint(model_filepath, save_best_only=True))
callbacks.append(CSVLogger(log_filepath))

for epoch in range(epochs):
    model.fit(train_gen,
              steps_per_epoch=steps_per_epoch/5,
              epochs=1,
              use_multiprocessing=True,
              callbacks = callbacks,
              verbose=1,

             )
    print('done fitting')
    loss,accuracy = model.evaluate(Sy_val,y_val_normalized)
    #print(loss,accuracy)

InternalError: CUDA runtime implicit initialization on GPU:0 failed. Status: all CUDA-capable devices are busy or unavailable